# 部分採用，擷取特徵

In [45]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

# 預先訓練好的模型 -- VGG16, 不含後三層(辨識層)
model = VGG16(weights='imagenet', include_top=False)

# 任意一張圖片，例如大象
img_path = './images/elephant.jpg'
# 載入圖檔，並縮放寬高為 (224, 224) 
img = image.load_img(img_path, target_size=(224, 224))
# 加一維，變成 (1, 224, 224, 3)，最後一維是色彩
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

In [46]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [47]:
model2 = VGG16(weights='imagenet', include_top=True)
model2.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [48]:
print(features)

[[[[ 0.          0.          0.         ...  0.          0.
     0.        ]
   [ 0.          0.         42.547802   ...  0.          0.
     0.        ]
   [ 1.0753415   0.         23.495632   ...  0.          0.
     0.        ]
   ...
   [ 0.          0.          0.         ...  0.          0.
     0.        ]
   [ 0.          0.          0.         ...  0.          0.
     0.        ]
   [ 0.          0.          0.         ...  0.          0.
     0.        ]]

  [[ 0.          0.         36.33888    ...  0.          0.
     3.4029133 ]
   [ 0.          0.         80.23636    ...  7.871857    0.
     0.        ]
   [ 0.          0.         48.751343   ...  0.          0.
     0.        ]
   ...
   [ 0.          0.          0.         ...  4.5813737   0.
     0.        ]
   [ 0.          0.          0.         ...  0.          0.
     0.        ]
   [ 0.          0.          0.         ...  0.          0.
     0.        ]]

  [[ 0.          0.          9.853498   ...  0.          0

In [49]:
print(features.shape)

(1, 7, 7, 512)


# 使用 cosine_similarity 比較特徵向量

In [50]:
from os import listdir
from os.path import isfile, join

# 讀取 images 目錄下所有圖檔
img_path = './images/'
image_files = np.array([f for f in listdir(img_path) if isfile(join(img_path, f)) and f[-3:] == 'jpg'])
image_files

array(['bird01.jpg', 'elephant.jpg', 'elephant2.jpg', 'input.jpg',
       'style.jpg', 'Tiger.jpg', 'Tiger2.jpg', 'Tiger3.jpg', '太陽花.jpg'],
      dtype='<U13')

In [51]:
# 每個圖檔經過預測，取得特徵向量
import numpy as np

X = np.array([])
# 合併每個圖檔的像素
for f in image_files:
    image_file = join(img_path, f)
    # 載入圖檔，並縮放寬高為 (224, 224) 
    img = image.load_img(image_file, target_size=(224, 224))
    # 加一維，變成 (1, 224, 224, 3)，最後一維是色彩
    img2 = image.img_to_array(img)
    img2 = np.expand_dims(img2, axis=0)
    if len(X.shape) == 1:
        X = img2
    else:
        X = np.concatenate((X, img2), axis=0)

X = preprocess_input(X)

# 預測
features = model.predict(X)

features.shape, X.shape

((9, 7, 7, 512), (9, 224, 224, 3))

In [54]:
# 使用 cosine_similarity 比較特徵向量
from sklearn.metrics.pairwise import cosine_similarity

features2 = features.reshape((features.shape[0], -1))

# 比較對象：Tiger3.jpg
no=-2
print(image_files[no])
similar_list = cosine_similarity(features2[no:no+1], features2[:no], dense_output=False)
print(np.sort(similar_list[0])[::-1])
image_files[:no][np.argsort(similar_list[0])[::-1]]

Tiger3.jpg
[0.31435788 0.26541096 0.12628621 0.09772755 0.07242107 0.04454595
 0.04227018]


array(['Tiger.jpg', 'Tiger2.jpg', 'style.jpg', 'elephant.jpg',
       'elephant2.jpg', 'input.jpg', 'bird01.jpg'], dtype='<U13')

In [53]:
# 比較對象：elephant.jpg
no=1
print(image_files[no])
similar_list = cosine_similarity(features2[no:no+1], features2[no+1:], dense_output=False)
print(np.sort(similar_list[0])[::-1])
image_files[no+1:][np.argsort(similar_list[0])[::-1]]

elephant.jpg
[0.29506823 0.1904265  0.1562427  0.09772753 0.07000628 0.03698231
 0.02468516]


array(['elephant2.jpg', 'Tiger2.jpg', 'Tiger.jpg', 'Tiger3.jpg',
       'style.jpg', '太陽花.jpg', 'input.jpg'], dtype='<U13')